In [7]:
#Imports
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from scipy.spatial import distance

# Training Classifier

In this part we will go trough training the classifier. We start of by importing the necessary data we prepared in the last section

In [4]:
eigenfaces_features = np.load('eigenfaces_features.npy')
emotion_labels = np.load('eigenfaces_labels.npy')

This loads the preprocessed eigenfaces features and labels from the corresponding `.npy` files using `np.load()`. Make sure the files are present in the same directory, if not so - go back to the previous notebook and run the code, which creates them.

In [5]:
cap = cv2.VideoCapture(0)  # 0 for the default camera, you can change it if needed

In [13]:
def detect_emotion(frame):
    # Preprocess the frame (e.g., face detection, cropping, resizing, etc.)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale for face detection

    # Perform face detection
    face_cascade = cv2.CascadeClassifier('datasets/haarcascade_frontalface_default.xml')  # Path to face detection cascade file
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return None  # No face detected, return None or handle the case appropriately

    # Extract the first face detected
    (x, y, width, height) = faces[0]
    face_roi = gray[y:y+height, x:x+width]  # Crop the face region from the frame

    # Resize the face ROI to a fixed size
    resized_face = cv2.resize(face_roi, (100, 100))  # Adjust the size as needed

    # Extract features from the resized face
    extracted_features = extract_features(resized_face)  # Implement your feature extraction method

    # Initialize variables for emotion detection
    min_distance = float('inf')
    predicted_label = None

    # Iterate over the eigenfaces features and compare distances
    for i in range(eigenfaces_features.shape[0]):
        distance = euclidean_distance(extracted_features, eigenfaces_features[i])

        if distance < min_distance:
            min_distance = distance
            predicted_label = emotion_labels[i]

    return predicted_label

In [9]:
def draw_square(frame, x, y, width, height, label):
    cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


In [14]:
while True:
    ret, frame = cap.read()

    # Perform emotion detection on the frame
    predicted_emotion = detect_emotion(frame)

    # Draw a square around the face and label the emotion
    draw_square(frame, x, y, width, height, predicted_emotion)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [12]:
cap.release()
cv2.destroyAllWindows()

This splits the dataset into training and testing sets using the `train_test_split()` function from `scikit-learn`. It takes the eigenfaces features (`features`) and labels (`labels`) as input and splits them into training and testing sets. Here, 80% of the data is used for training, and 20% is used for testing.

In [6]:
def euclidean_distance(feature_vector1, feature_vector2):
    return distance.euclidean(feature_vector1, feature_vector2)

Now we create an instance of the Support Vector Machine (`SVM`) classifier with a linear kernel. It then trains the classifier using the training data (`X_train` and `y_train`) using the `fit()` method.

In [17]:
# Load the face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

Now we need to load the pre-trained face detection cascade classifier `XML` file using the `cv2.CascadeClassifier()` function. The `cv2.data.haarcascades` provides the path to the data directory where the pre-trained classifiers are located. The XML file used here is `haarcascade_frontalface_default.xml`.

In [8]:
# Start the video capture
cap = cv2.VideoCapture(0)

After that we start the video capture by initializing a `VideoCapture` object cap with index `0`, which represents the default camera device.

In [18]:
while True:
    # Read the video frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterate over detected faces
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Extract the face region
        face_roi = gray[y:y + h, x:x + w]

        # Preprocess the face region
        face_roi = cv2.resize(face_roi, (100, 100))
        face_roi = face_roi / 255.0

        # Flatten the face ROI
        face_roi = face_roi.flatten()

        # Perform eigenface extraction
        eigenface = np.dot(face_roi, eigenfaces.T)

        # Make emotion prediction
        prediction = classifier.predict([eigenface])

        # Map the predicted label index to the corresponding emotion label
        predicted_emotion = emotion_labels[prediction[0]]

        # Draw the predicted emotion label on the face rectangle
        label = f"Emotion: {predicted_emotion}"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Facial Emotion Recognition', frame)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [19]:
# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()